In [1]:
from utils.chesscom_downloader import ChesscomDownloader
import matplotlib.pyplot as plt

downloader = ChesscomDownloader(timeout=20.0, sleep_sec=0.2)
username = "joeshift"
df = downloader.load_from_cache(username, timezone="Europe/Berlin")

2025-10-04 15:53:37.957 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-04 15:53:38.740 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [3]:
df["pgn"]

0      [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...
1      [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...
2      [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...
3      [Event "Play vs Coach"]\n[Site "Chess.com"]\n[...
4      [Event "Play vs Coach"]\n[Site "Chess.com"]\n[...
                             ...                        
271    [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...
272    [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...
273    [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...
274    [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...
275    [Event "Live Chess"]\n[Site "Chess.com"]\n[Dat...
Name: pgn, Length: 276, dtype: object

In [ ]:
import io
import chess.pgn
import re

import pandas as pd

_CLK_RE = re.compile(r"\[%clk\s+([0-9:.]+)\]")

def parse_pgn_with_clock_safe(row: str):
    pgn_text = str(row) if pd.notna(row) else ""

    # Parse the first game in the PGN string
    game = chess.pgn.read_game(io.StringIO(pgn_text))
    if game is None:
        return None

    # Header fields
    headers = dict(game.headers)

    # Traverse moves
    moves = []
    board = game.board()
    for node in game.mainline():
        move = node.move
        san = board.san(move)

        # Extract clock time from comment (if present)
        comment = node.comment
        clk = None
        if comment:
            m = _CLK_RE.search(comment)
            if m:
                clk = m.group(1)

        moves.append({"ply": len(moves)+1, "move": san, "clock": clk})
        board.push(move)

    return headers, moves

df[["pgn_headers", "pgn_moves"]] = df["pgn"].apply(
    lambda s: pd.Series(parse_pgn_with_clock_safe(s))
)

In [ ]:
import io, re, json
import pandas as pd
import chess.pgn

_CLK_RE = re.compile(r"\[%clk\s+([0-9:.]+)\]")

def parse_pgn_min(pgn_text: str) -> dict:
    game = chess.pgn.read_game(io.StringIO(pgn_text or ""))
    if not game:
        return {
            "eco": None, "eco_url": None,
            "moves_normalized": None,
            "moves_san_json": "[]", "clocks_json": "[]",
            "n_plies": 0
        }

    hdr = game.headers
    eco = hdr.get("ECO")

    board = game.board()
    sans, clocks = [], []
    for node in game.mainline():
        san = board.san(node.move)
        board.push(node.move)
        cmt = node.comment or ""
        m = _CLK_RE.search(cmt)
        sans.append(san)
        clocks.append(m.group(1) if m else None)

    moves_normalized = " ".join(
        f"{(i//2)+1}. {m}" if i % 2 == 0 else m
        for i, m in enumerate(sans)
    )

    return {
        "eco": eco,
        "moves_normalized": moves_normalized,
        "moves_san_json": json.dumps(sans),
        "clocks_json": json.dumps(clocks),
        "n_plies": len(sans),
    }

# df: your main table that already has chess.com fields (username, ratings, result, time_class, time_control, game_url, etc.)
# Add only the PGN-derived minimal columns, then drop the raw PGN.

pgn_cols = df["pgn"].apply(parse_pgn_min).pipe(lambda s: pd.DataFrame(s.tolist()))
df = pd.concat([df.drop(columns=["pgn"]), pgn_cols], axis=1)

# Optional: if API already gives an eco/eco_url column, prefer the non-null one and drop duplicates.
for col in ["eco", "eco_url"]:
    api_col = f"{col}_api"
    if api_col in df.columns:
        df[col] = df[api_col].fillna(df[col])
        df = df.drop(columns=[api_col])

# Save single-table Parquet
#df.to_parquet("chess_games_flat.parquet", engine="pyarrow", index=False)
df

,end_time,username,opponent_username,user_played_as,user_result,user_result_simple,opponent_result,user_rating,opponent_rating,rated,...,opening_id,month_name,pgn_headers,pgn_moves,eco,eco_url,moves_normalized,moves_san_json,clocks_json,n_plies
0,2020-07-10 23:04:33+00:00,JoeShift,BryceMattos,white,win,win,resigned,1078,954,True,...,2665,July,"{'Event': 'Live Chess', 'Site': 'Chess.com', '...","[{'ply': 1, 'move': 'e4', 'clock': '0:05:04.9'...",C30,https://www.chess.com/openings/Kings-Gambit-2....,1. e4 e5 2. f4 d6 3. Bc4 Nf6 4. Nc3 a6 5. Nf3 ...,"[""e4"", ""e5"", ""f4"", ""d6"", ""Bc4"", ""Nf6"", ""Nc3"", ...","[""0:05:04.9"", ""0:05:04.9"", ""0:05:08.4"", ""0:05:...",39
1,2020-07-10 23:22:29+00:00,JoeShift,rhasc,black,win,win,resigned,1254,1194,True,...,2665,July,"{'Event': 'Live Chess', 'Site': 'Chess.com', '...","[{'ply': 1, 'move': 'e4', 'clock': '0:05:04.9'...",B20,https://www.chess.com/openings/Sicilian-Defens...,1. e4 c5 2. Bc4 Nc6 3. Bxf7+ Kxf7 4. Qh5+ g6 5...,"[""e4"", ""c5"", ""Bc4"", ""Nc6"", ""Bxf7+"", ""Kxf7"", ""Q...","[""0:05:04.9"", ""0:05:02.5"", ""0:05:07.6"", ""0:04:...",116
2,2023-06-16 14:50:59+00:00,JoeShift,LucBrasi,white,win,win,timeout,1254,678,False,...,2665,June,"{'Event': 'Live Chess', 'Site': 'Chess.com', '...","[{'ply': 1, 'move': 'e4', 'clock': '0:10:00'},...",C30,https://www.chess.com/openings/Kings-Gambit-De...,1. e4 e5 2. f4 Nc6 3. Bc4 d5 4. Bxd5 f6 5. f5 ...,"[""e4"", ""e5"", ""f4"", ""Nc6"", ""Bc4"", ""d5"", ""Bxd5"",...","[""0:10:00"", ""0:10:00"", ""0:09:58.8"", ""0:09:46.4...",61
3,2025-09-04 17:02:43+00:00,JoeShift,Coach-David,white,win,win,checkmated,800,900,False,...,758,September,"{'Event': 'Play vs Coach', 'Site': 'Chess.com'...","[{'ply': 1, 'move': 'e4', 'clock': None}, {'pl...",B00,https://www.chess.com/openings/Nimzowitsch-Def...,1. e4 Nc6 2. f4 h5 3. Nf3 Nd4 4. Nxd4 e5 5. Nf...,"[""e4"", ""Nc6"", ""f4"", ""h5"", ""Nf3"", ""Nd4"", ""Nxd4""...","[null, null, null, null, null, null, null, nul...",83
4,2025-09-04 17:17:44+00:00,JoeShift,Coach-David,white,win,win,checkmated,800,1200,False,...,1133,September,"{'Event': 'Play vs Coach', 'Site': 'Chess.com'...","[{'ply': 1, 'move': 'e4', 'clock': None}, {'pl...",B21,https://www.chess.com/openings/Sicilian-Defens...,1. e4 c5 2. f4 g6 3. Nf3 d6 4. Bc4 b5 5. Bxb5+...,"[""e4"", ""c5"", ""f4"", ""g6"", ""Nf3"", ""d6"", ""Bc4"", ""...","[null, null, null, null, null, null, null, nul...",103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,2025-10-02 20:45:59+00:00,JoeShift,Paccanice,black,resigned,loss,win,843,829,True,...,2665,October,"{'Event': 'Live Chess', 'Site': 'Chess.com', '...","[{'ply': 1, 'move': 'e4', 'clock': '0:04:59.1'...",B30,https://www.chess.com/openings/Sicilian-Defens...,1. e4 c5 2. Nf3 Nc6 3. Bc4 g6 4. Ng5 e6 5. Qf3...,"[""e4"", ""c5"", ""Nf3"", ""Nc6"", ""Bc4"", ""g6"", ""Ng5"",...","[""0:04:59.1"", ""0:04:56.9"", ""0:04:55.8"", ""0:04:...",23
272,2025-10-02 20:52:04+00:00,JoeShift,GomezDanilo,black,win,win,abandoned,851,821,True,...,2971,October,"{'Event': 'Live Chess', 'Site': 'Chess.com', '...","[{'ply': 1, 'move': 'd4', 'clock': '0:04:58.1'...",D00,https://www.chess.com/openings/Queens-Pawn-Ope...,1. d4 d5 2. e3 Nf6 3. c3 e6 4. f3 b6 5. g4 h6 ...,"[""d4"", ""d5"", ""e3"", ""Nf6"", ""c3"", ""e6"", ""f3"", ""b...","[""0:04:58.1"", ""0:04:57.4"", ""0:04:58"", ""0:04:55...",46
273,2025-10-03 17:06:10+00:00,JoeShift,Kman3340,white,win,win,resigned,859,847,True,...,2665,October,"{'Event': 'Live Chess', 'Site': 'Chess.com', '...","[{'ply': 1, 'move': 'e4', 'clock': '0:04:58.6'...",C22,https://www.chess.com/openings/Center-Game-Acc...,1. e4 e5 2. d4 exd4 3. Qxd4 Nc6 4. Qe3 d6 5. B...,"[""e4"", ""e5"", ""d4"", ""exd4"", ""Qxd4"", ""Nc6"", ""Qe3...","[""0:04:58.6"", ""0:04:59.4"", ""0:04:57.8"", ""0:04:...",37
274,2025-10-03 19:45:20+00:00,JoeShift,ackerman249,white,win,win,resigned,867,842,True,...,2665,October,"{'Event': 'Live Chess', 'Site': 'Chess.com', '...","[{'ply': 1, 'move':

In [11]:
df["pgn_moves"]

0      [{'ply': 1, 'move': 'e4', 'clock': '0:05:04.9'...
1      [{'ply': 1, 'move': 'e4', 'clock': '0:05:04.9'...
2      [{'ply': 1, 'move': 'e4', 'clock': '0:10:00'},...
3      [{'ply': 1, 'move': 'e4', 'clock': None}, {'pl...
4      [{'ply': 1, 'move': 'e4', 'clock': None}, {'pl...
                             ...                        
271    [{'ply': 1, 'move': 'e4', 'clock': '0:04:59.1'...
272    [{'ply': 1, 'move': 'd4', 'clock': '0:04:58.1'...
273    [{'ply': 1, 'move': 'e4', 'clock': '0:04:58.6'...
274    [{'ply': 1, 'move': 'e4', 'clock': '0:04:59.6'...
275    [{'ply': 1, 'move': 'e4', 'clock': '0:04:59.8'...
Name: pgn_moves, Length: 276, dtype: object